<h1>Extracting results</h1>
<p>Here we extract data required to plot results from the data generated by 
<a href="https://github.com/monarch-initiative/negativeExampleSelection/blob/main/scripts/runSTRING.py">runString.py</a> and
    <a href="https://github.com/monarch-initiative/negativeExampleSelection/blob/main/scripts/runSli.py">runSlipy</a>. Note that we also provide scripts for running these two Python scripts in a SLURM environment,
 <a href="https://github.com/monarch-initiative/negativeExampleSelection/blob/main/scripts/runStringWithSlurm.py">runStringWithSlurm.sh</a> and
    <a href="https://github.com/monarch-initiative/negativeExampleSelection/blob/main/scripts/runSliWithSlurm.sh">runSliWithSlurm.sh</a> </p>
<p>The result of analysis with these scripts are output files
  <a href="https://github.com/monarch-initiative/negativeExampleSelection/blob/main/scripts/sli_only_results.csv">sli_only_results.csv</a>  which are used in this Notebook to calculate the means and standard deviations for the computational experiments.
</p>

In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None 
from glob import glob
import json

<h3>collecting data</h3>
<p>The following function renames some columns, recodes some True/False columns using string values, and outputs a subset of columns into a CSV file that can be used for plotting</p>

In [32]:
def extract_core_results(df):
    df["evaluation_negative_sampling_method"] = ["DANS" if use_scale_free_distribution else "UNS"
        for use_scale_free_distribution in df.use_scale_free_distribution]
    df["model_negative_sampling_method"] = [ "DANS" if use_scale_free_distribution else "UNS"
          for use_scale_free_distribution in df["('model_parameters', 'use_scale_free_distribution')"]                                  ]
    columns = ["evaluation_mode", "features_names","evaluation_negative_sampling_method",
               "model_negative_sampling_method", "accuracy",  "balanced_accuracy", 
               "false_discovery_rate","matthews_correlation_coefficient",
           "precision", "recall", "specificity", "f1_score", "auroc", "auprc"]
    return df[columns].copy()

<H2>Aggregating Results</H2>
<p>Here, we calculate the mean and standard error of the
    balanced accuracy, false discovery rate, matthews correlation coefficient,
    F1 score, AUROC, and AUPRC.</p>

In [33]:
def get_mean_and_sd(df):
    """
    df should be one of string_results or sli_results
    """
    # We are just interested in the following seven graph/random walk methods
    graph_methods = {'First-order LINE', 'DeepWalk SkipGram', 'Walklets CBOW', 'HOPE',
                 'Second-order LINE','DeepWalk CBOW',    'Walklets SkipGram'}
    df = df[df['features_names'].isin(graph_methods)]
    # Rename columns for conciseness
    df =  df.rename(columns={"evaluation_negative_sampling_method": "evaluation", 
                   "evaluation_mode": "mode","features_names":"methods",
                    "model_negative_sampling_method": "model_neg_sampling"}, errors="raise")
    # Define the columns that we want to calculate mean and sem for
    evaluation_d = {'balanced_accuracy':['mean','std'],
               'false_discovery_rate':['mean','std'], 
               'matthews_correlation_coefficient':['mean','std'], 
               'f1_score':['mean','std'], 
               'auroc':['mean','std'], 
               'auprc':['mean','std']}
    # Calculate mean and standard dev
    df2 =  df.groupby(['methods','mode','evaluation','model_neg_sampling']).agg(evaluation_d).reset_index()
    # Make a new column for convenience in planning
    df2["approach"] = df2["evaluation"] + " (" + df2["mode"] + ")"
    return df2

<h3>Rename columns from tuples to simple strings for readability</h3>

In [34]:
def rename_columns(df):
   
    columns = ["methods",'mode','evaluation', "model_neg_sampling",
           "balanced_acc.mean","balanced_acc.std", 
           "FDR.mean", "FDR.std", 
           "MCC.mean", "MCC.std", 
           "F1.mean", "F1.std", 
           "AUROC.mean", "AUROC.std", 
           "AUPRC.mean","AUPRC.std", 
           "approach"]
    df.columns = columns
    return df

<h3>SLI synthetic lethality results</h3>

In [35]:
df = pd.read_csv("sli_only_results_jul23.tsv", sep='\t')

In [36]:
sli_results = extract_core_results(df)

In [37]:
sli_results.head()

,evaluation_mode,features_names,evaluation_negative_sampling_method,model_negative_sampling_method,accuracy,balanced_accuracy,false_discovery_rate,matthews_correlation_coefficient,precision,recall,specificity,f1_score,auroc,auprc
0,train,First-order LINE,DANS,DANS,0.676118,0.676118,0.312645,0.352871,0.687355,0.646129,0.706107,0.666105,0.736399,0.706748
1,test,First-order LINE,DANS,DANS,0.591653,0.591653,0.384298,0.187399,0.615702,0.487725,0.695581,0.544292,0.645766,0.637471
2,train,First-order LINE,DANS,UNS,0.771810,0.771810,0.206937,0.545056,0.793063,0.735551,0.808070,0.763225,0.823869,0.837614
3,test,First-order LINE,DANS,UNS,0.698854,0.698854,0.238710,0.409574,0.761290,0.579378,0.818331,0.657993,0.731430,0.748987
4,train,First-order LINE,DANS,DANS,0.681025,0.681025,0.294808,0.364588,0.705192,0.622137,0.739913,0.661066,0.744830,0.727736


In [38]:
sli_stats = get_mean_and_sd(sli_results)

In [39]:
sli_stats.head()

methods   mode evaluation model_neg_sampling balanced_accuracy  \
                                                                   mean   
0  DeepWalk CBOW   test       DANS               DANS          0.613175   
1  DeepWalk CBOW   test       DANS                UNS          0.803110   
2  DeepWalk CBOW   test        UNS               DANS          0.619108   
3  DeepWalk CBOW   test        UNS                UNS          0.866162   
4  DeepWalk CBOW  train       DANS               DANS          0.806543   

            false_discovery_rate           matthews_correlation_coefficient  \
        std                 mean       std                             mean   
0  0.014518             0.286633  0.034964                         0.256424   
1  0.017240             0.181817  0.026000                         0.607548   
2  0.024639             0.243850  0.039998                         0.281914   
3  0.015686             0.061168  0.021558                         0.742785   
4  0.029446             0.167236  0.032533                         0.615213   

             f1_score               auroc               auprc            \
        std      mean       std      mean       std      mean       std   
0  0.033425  0.495685  0.029277  0.679624  0.034233  0.674321  0.028182   
1  0.034897  0.798539  0.017842  0.877389  0.016808  0.877670  0.016867   
2  0.051737  0.477817  0.049237  0.702873  0.040953  0.698838  0.033761   
3  0.031252  0.854050  0.017742  0.945937  0.016853  0.949158  0.015656   
4  0.059065  0.798591  0.030886  0.884297  0.030673  0.881733  0.024366   

       approach  
                 
0   DANS (test)  
1   DANS (test)  
2    UNS (test)  
3    UNS (test)  
4  DANS (train)

In [40]:
sli_stats = rename_columns(sli_stats)
sli_stats.head()

,methods,mode,evaluation,model_neg_sampling,balanced_acc.mean,balanced_acc.std,FDR.mean,FDR.std,MCC.mean,MCC.std,F1.mean,F1.std,AUROC.mean,AUROC.std,AUPRC.mean,AUPRC.std,approach
0,DeepWalk CBOW,test,DANS,DANS,0.613175,0.014518,0.286633,0.034964,0.256424,0.033425,0.495685,0.029277,0.679624,0.034233,0.674321,0.028182,DANS (test)
1,DeepWalk CBOW,test,DANS,UNS,0.803110,0.017240,0.181817,0.026000,0.607548,0.034897,0.798539,0.017842,0.877389,0.016808,0.877670,0.016867,DANS (test)
2,DeepWalk CBOW,test,UNS,DANS,0.619108,0.024639,0.243850,0.039998,0.281914,0.051737,0.477817,0.049237,0.702873,0.040953,0.698838,0.033761,UNS (test)
3,DeepWalk CBOW,test,UNS,UNS,0.866162,0.015686,0.061168,0.021558,0.742785,0.031252,0.854050,0.017742,0.945937,0.016853,0.949158,0.015656,UNS (test)
4,DeepWalk CBOW,train,DANS,DANS,0.806543,0.029446,0.167236,0.032533,0.615213,0.059065,0.798591,0.030886,0.884297,0.030673,0.881733,0.024366,DANS (train)


In [41]:
sli_stats.to_csv("sli_stats.csv")

<h3>STRING Protein-Protein Association data</h3>